#### 1.子公司数据归属地转换

In [1]:
import pandas as pd
import numpy as np

In [2]:
company = pd.read_excel('../../file/subject/chapter3/company_all.xlsx')
len(company)

57432

#### 1.1抽出一部分作为样本进行处理

In [3]:
company.iloc[78:81,:][['维度','经度','地址']]

,维度,经度,地址
78,NaN,NaN,湖南省澧县王家厂镇建设街居委会渔场直属村008号
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场


#### 1.2填充没有经纬度的行

In [10]:
import requests
import json
import re

def company_lat_lng(x):
    if x['维度'] == np.nan:
        address = x['地址']
        url = 'http://api.map.baidu.com/geocoding/v3/?address=' + address + '&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&callback=showLocation'
        data = requests.get(url).text
        js = re.match('.*?\((.*?)\).*?',data).group(1)
        js = json.loads(js)['result']['location']
        return js

In [5]:
company1 = company.iloc[78:81,:][['维度','经度','地址']]

In [14]:
company1.apply(company_lat_lng,axis=1)

78    None
79    None
80    None
dtype: object

In [29]:
#全都没有获取数据，排查发现表格中的Nan并非numpy中的nan
company1['维度'][78] == np.nan

False

In [6]:
#将空值全部替换为0
company2 = company1.fillna(0).copy()

In [7]:
company2['维度'][78]  == 0

True

In [45]:
def company_lat_lng(x):
    if x['维度'] == 0:
        address = x['地址']
        url = 'http://api.map.baidu.com/geocoding/v3/?address=' + address + '&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&callback=showLocation'
        data = requests.get(url).text
        js = re.match('.*?\((.*?)\).*?',data).group(1)
        js = json.loads(js)['result']['location']
        return js

In [8]:
#成功判断并获取数据
company2.apply(company_lat_lng,axis=1)

78    {'lng': 111.53989881872957, 'lat': 29.78702709...
79                                                    0
80                                                    0
dtype: object

#### 1.3优化代码

In [4]:
import requests
import json
import re

def company_lat_lng(x):
    try:
        if x['维度'] == 0:
            address = x['地址']
            url = 'http://api.map.baidu.com/geocoding/v3/?address=' + address + '&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&callback=showLocation'
            data = requests.get(url).text
            js = re.match('.*?\((.*?)\).*?',data).group(1)
            js = json.loads(js)['result']['location']
            return js
        else:
            return 0
    except:
        print(x['地址'])

In [9]:
company2.apply(company_lat_lng,axis=1)

78    {'lng': 111.53989881872957, 'lat': 29.78702709...
79                                                    0
80                                                    0
dtype: object

In [5]:
company.fillna(0,inplace=True)

In [ ]:
company['lat_lng'] = company[['维度','地址']].apply(company_lat_lng,axis=1)

In [8]:
company3 = company.iloc[78:81,:][['维度','经度','地址','lat_lng']].copy()
company3

,维度,经度,地址,lat_lng
78,0.000000,0.000000,湖南省澧县王家厂镇建设街居委会渔场直属村008号,"{'lng': 111.53989881872957, 'lat': 29.78702709..."
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室,0
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场,0


In [9]:
company3.lat_lng = company3.apply(lambda x:x.lat_lng if x['维度'] == 0 else np.nan,axis=1)

In [13]:
company3

,维度,经度,地址,lat_lng
78,0.000000,0.000000,湖南省澧县王家厂镇建设街居委会渔场直属村008号,"{'lng': 111.53989881872957, 'lat': 29.78702709..."
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室,NaN
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场,NaN


In [33]:
company3.维度 = company3.apply(lambda x:x.lat_lng['lat'] if type(x.lat_lng) == dict else x.维度,axis=1)
company3.经度 = company3.apply(lambda x:x.lat_lng['lng'] if type(x.lat_lng) == dict else x.经度,axis=1)

In [55]:
company3

,维度,经度,地址,lat_lng
78,29.787027,111.539899,湖南省澧县王家厂镇建设街居委会渔场直属村008号,"{'lng': 111.53989881872957, 'lat': 29.78702709..."
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室,NaN
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场,NaN


In [35]:
company.维度 = company.apply(lambda x:x.lat_lng['lat'] if type(x.lat_lng) == dict else x.维度,axis=1)
company.经度 = company.apply(lambda x:x.lat_lng['lng'] if type(x.lat_lng) == dict else x.经度,axis=1)

In [36]:
company.iloc[78:81,:][['维度','经度','地址','lat_lng']]

,维度,经度,地址,lat_lng
78,29.787027,111.539899,湖南省澧县王家厂镇建设街居委会渔场直属村008号,"{'lng': 111.53989881872957, 'lat': 29.78702709..."
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室,0
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场,0


#### 1.4通过经纬度获取归属地

In [60]:
def company_region(lat_lnt):
    try:
        if lat_lnt[0] != 0:
            url = 'http://api.map.baidu.com/reverse_geocoding/v3/\
?ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&output=json&coordtype=wgs84ll&location={},{}'.format(lat_lnt[0],lat_lnt[1])
            data = requests.get(url).text
            data = json.loads(data)
            ls = []
            ls.append(data['result']['addressComponent']['province'])
            ls.append(data['result']['addressComponent']['city'])
            ls.append(data['result']['addressComponent']['district'])
            return ls
        else:
            return 0
    except:
        print('error')

In [61]:
company3[['维度','经度']].apply(company_region,axis=1)

78            [湖南省, 常德市, 澧县]
79          [上海市, 上海市, 浦东新区]
80    [云南省, 红河哈尼族彝族自治州, 蒙自市]
dtype: object

In [ ]:
company['region'] = company[['维度','经度']].apply(company_region,axis=1)

In [ ]:
company.to_excel(r'../../file/subject/chapter3/company_info_all03.xlsx')

In [63]:
company = pd.read_excel(r'../../file/subject/chapter3/company_info_all03.xlsx')

In [93]:
company4 = company.iloc[78:83,:].copy()

In [94]:
#将归属地的字符串转换为列表
re.findall('\'(.*?)\'',company4.region[78])

['黑龙江省', '哈尔滨市', '道里区']

In [95]:
company4.region = company4.region.apply(lambda x:re.findall('\'(.*?)\'',x))

In [96]:
company4.region[78][0]

'黑龙江省'

In [123]:
company.region = company.region.apply(lambda x:re.findall('\'(.*?)\'',x) if x != 0 else 0)

In [125]:
company.region[0][0]

'上海市'

In [126]:
company['province'] = company.region.apply(lambda x:x[0] if x != 0 else 0)

In [127]:
company['city'] = company.region.apply(lambda x:x[1] if x != 0 else 0)

In [129]:
company.columns

Index(['Unnamed: 0', '纬度', '经度', '经营范围', '公司特点', '公司地址', '公司网址', '公司高管',
       '公司名称', '上市公司名称', '公司状态', '注册资本', '上市公司投资比例', 'lat_lng', 'region',
       'province', 'city'],
      dtype='object')

In [130]:
company_son = company[['公司名称', '上市公司名称', '公司状态', '注册资本','上市公司投资比例','province', 'city']].copy()

In [131]:
company_son.head()

,公司名称,上市公司名称,公司状态,注册资本,上市公司投资比例,province,city
0,上海盛美投资咨询有限公司,甘肃亚盛实业(集团)股份有限公司,存续（在营、开业、在册）,500.0000(万人民币),80%,上海市,上海市
1,甘肃证券有限责任公司,甘肃亚盛实业(集团)股份有限公司,吊销，未注销,"10,000.0000(万元人民币)",20%,甘肃省,兰州市
2,酒泉市万客隆商贸有限责任公司,甘肃亚盛实业(集团)股份有限公司,吊销，未注销,116.0000(万元人民币),0%,甘肃省,酒泉市
3,甘肃亚盛投资咨询有限公司,甘肃亚盛实业(集团)股份有限公司,注销（注销日期：2011年9月28日）,"1,000.0000(万元人民币)",100%,甘肃省,兰州市
4,甘肃力维氯碱化工有限公司,甘肃亚盛实业(集团)股份有限公司,吊销，未注销,"3,330.0000(万元人民币)",75.08%,甘肃省,兰州市
